In [31]:
from keras.datasets import imdb
from nltk.tokenize.treebank import TreebankWordTokenizer
import numpy as np

In [18]:
vocabulary_size = 5000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words = vocabulary_size)
print('Loaded dataset with {} training samples, {} test samples'.format(len(X_train), len(X_test)))

Loaded dataset with 25000 training samples, 25000 test samples


In [19]:
from keras.preprocessing import sequence
max_words = 500
X_train = sequence.pad_sequences(X_train, maxlen=max_words)
X_test = sequence.pad_sequences(X_test, maxlen=max_words)

In [21]:
X_train.shape

(25000, 500)

In [5]:
from keras import Sequential
from keras.layers import Embedding, LSTM, Dense, Bidirectional
embedding_size=32
model=Sequential()
model.add(Embedding(vocabulary_size, embedding_size, input_length=max_words))
model.add(Bidirectional(LSTM(100)))
model.add(Dense(1, activation='sigmoid'))
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 500, 32)           160000    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 200)               106400    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 201       
Total params: 266,601
Trainable params: 266,601
Non-trainable params: 0
_________________________________________________________________
None


In [6]:
model.compile(loss='binary_crossentropy', 
             optimizer='adam', 
             metrics=['accuracy'])

In [7]:
model.load_weights("wsbidirectional")

In [16]:
batch_size = 64
num_epochs = 3
X_valid, y_valid = X_train[:batch_size], y_train[:batch_size]
X_train2, y_train2 = X_train[batch_size:], y_train[batch_size:]
model.fit(X_train2, y_train2, validation_data=(X_valid, y_valid), batch_size=batch_size, epochs=num_epochs)

Train on 24936 samples, validate on 64 samples
Epoch 1/3
24936/24936 [==============================] - 499s 20ms/step - loss: 0.4865 - acc: 0.7635 - val_loss: 0.3119 - val_acc: 0.8906
Epoch 2/3
24936/24936 [==============================] - 467s 19ms/step - loss: 0.3173 - acc: 0.8699 - val_loss: 0.2865 - val_acc: 0.8750
Epoch 3/3
24936/24936 [==============================] - 470s 19ms/step - loss: 0.2674 - acc: 0.8959 - val_loss: 0.2849 - val_acc: 0.8750


In [8]:
score = model.evaluate(X_test, y_test)

25000/25000 [==============================] - 100s 4ms/step


In [9]:
print('accuracy: ', score[1])

accuracy:  0.85872


In [44]:
word_to_id = imdb.get_word_index()
word_to_id = {k:(v+3) for k,v in word_to_id.items()}
word_to_id["<PAD>"] = 0
word_to_id["<START>"] = 1
word_to_id["<UNK>"] = 2

def senToVe(sent):
    tk = TreebankWordTokenizer()
    sent = tk.tokenize(sent)
    sent = [word_to_id[sent[-499 + len(sent) - 1 + i]] if i > 499 - len(sent) else 0 for i in range(500)]
    sent = np.reshape(sent, (1 , 500))
    return sent

In [75]:
sent = "this video was not awful"
model.predict(senToVe(sent))

array([[0.07116483]], dtype=float32)